In [15]:
### Import packages

import pandas as pd
import geopandas as gpd
from pysal.explore import esda
from pysal.lib import weights

In [16]:
### Load data

# Specify top-n friends for CGFR
topnfriends = 25
cgfr = "cgfr_" + str(topnfriends)

# Load cross-gender ties dataset
cgt_filepath = "data sources/gadm2_cgfr.csv"
cgt_index = "region_id"
country = "DE"
cgt = pd.read_csv(cgt_filepath, index_col=cgt_index)
cgt = cgt[(cgt["country"] == country)] # Filter for country-specific data

# Load level 2 administrative boundaries dataset
admin2_filepath = "data sources/gadm41_DEU.gpkg"
admin2_layer = "ADM_ADM_2"
admin2_index = "GID_2"
admin2 = gpd.read_file(admin2_filepath, layer=admin2_layer)
admin2 = admin2.set_index(admin2_index)
admin2 = admin2[(admin2["ENGTYPE_2"] == "District")] # Filter out non-district data

In [17]:
### Perform spatial analysis using Moran's I

# Join geometry to attributes
db = admin2.join(cgt)

# Create spatial weights matrix
w = weights.KNN.from_dataframe(db, k=8)
w.transform = "R"

# Compute spatial lag
cgfr_lag = cgfr + "_lag"
db[cgfr_lag] = weights.lag_spatial(
    w, db[cgfr]
)
# Standardize CGFR by subtracting its mean
cgfr_std = cgfr + "_std"
cgfr_lag_std = cgfr_lag + "_std"
db[cgfr_std] = db[cgfr] - db[cgfr].mean()
db[cgfr_lag_std] = weights.lag_spatial(
    w, db[cgfr_std]
)

# Calculate Moran's I
moran = esda.moran.Moran(db[cgfr], w)
print(f"Moran's I: {moran.I}")
print(f"Moran's I p-value: {moran.p_sim}")

# Calculate local Moran's I
lisa = esda.moran.Moran_Local(db[cgfr], w)
# Assign pseudo P-values and determine their significance at 5% CL
print(f"Percent of polygons with statistically significant LISAs: {(lisa.p_sim < 0.05).sum() * 100 / len(lisa.p_sim)}")
db["p-sim"] = lisa.p_sim
sig = 1 * (lisa.p_sim < 0.05)
db["sig"] = sig
# Assign clustering values to features
spots = lisa.q * sig
spots_labels = {
    0: "Non-Significant",
    1: "HH",
    2: "LH",
    3: "LL",
    4: "HL",
}
db["labels"] = pd.Series(
    spots,
    index=db.index
).map(spots_labels)

Moran's I: 0.6297080121506486
Moran's I p-value: 0.001
Percent of polygons with statistically significant LISAs: 45.77114427860697


In [18]:
### Export dataset

# Filter dataset for specific fields
db = db[[
    "NAME_1", "NAME_2", cgfr, cgfr_lag, cgfr_std, cgfr_lag_std, "p-sim", "labels", "geometry"
]]
# Rename dataset fields for export use
db_renamed = db.rename(columns={
    "NAME_1": "State",
    "NAME_2": "District",
    cgfr: "Cross-Gender Friending Ratio (CGFR)",
    cgfr_lag: "CGFR spatial lag",
    cgfr_std: "Standardized CGFR",
    cgfr_lag_std: "Standardized CGFR spatial lag",
    "p-sim": "Pseudo-p-value",
    "labels": "Cluster type"
})
# Export filtered dataset
db_renamed.to_file("data outputs/cgfr_germany_localmoransi.geojson", driver="GeoJSON")